In [13]:
import torch
import numpy as np
import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS

import pandas as pd
from matplotlib import pyplot as plt
import random
import utils as ut

In [8]:
#create some fake players
names=["Fedo", "Nour", "MaLong", "Mauro", "TimoBoll", "JanOveWaldner"]
with open('fake_players.txt', 'w') as f:
    for name in names:
        f.write(name)
        f.write('\n')
IDs = list(range(1, len(names)+1))
n_players=len(names)

In [9]:
#generate some fake scores
n_games=50
winners=[]
losers=[]
points=[]
for i in range(n_games):
    winner,loser=random.sample(names,2)
    winners.append(winner)
    losers.append(loser)
    points.append(random.randrange(10))

In [11]:
winner_ids=torch.tensor([0,4,2,7,4,2,7,5,1,4,2,3,7,5,6,3])
loser_ids=torch.tensor([1,2,3,5,3,6,5,3,2,1,4,6,3,2,4,5])
loser_scores=torch.tensor([3,2,5,7,3,10,7,6,8,4,3,5,6,7,3,2])

In [14]:
# Run inference using NUTS (No-U-Turn Sampler) in MCMC
nuts_kernel = NUTS(model)
mcmc_run = MCMC(nuts_kernel, num_samples=100, warmup_steps=10)
mcmc_run.run(winner_ids, loser_ids, loser_scores)

# Get posterior samples
posterior_samples = mcmc_run.get_samples()




Sample: 100%|██| 110/110 [03:39,  1.99s/it, step size=1.17e-04, acc. prob=0.410]


In [15]:
# Print the posterior mean and standard deviation of player strengths
for i in range(len(np.unique(winner_ids))):
    player = "Player {}".format(i+1)
    print(player)
    print("Posterior mean of strength:", posterior_samples['strength'][:, i].mean().item())
    print("Posterior std dev of strength:", posterior_samples['strength'][:, i].std().item())
    print()

Player 1
Posterior mean of strength: -1.7637139558792114
Posterior std dev of strength: 0.07239000499248505

Player 2
Posterior mean of strength: 1.139736533164978
Posterior std dev of strength: 0.18123143911361694

Player 3
Posterior mean of strength: -1.519075870513916
Posterior std dev of strength: 0.07126716524362564

Player 4
Posterior mean of strength: 0.5709871053695679
Posterior std dev of strength: 0.053876399993896484

Player 5
Posterior mean of strength: 0.4991966187953949
Posterior std dev of strength: 0.05990365520119667

Player 6
Posterior mean of strength: 0.3477221429347992
Posterior std dev of strength: 0.06233220174908638

Player 7
Posterior mean of strength: -0.5613820552825928
Posterior std dev of strength: 0.05161304399371147

Player 8
Posterior mean of strength: 0.517486572265625
Posterior std dev of strength: 0.11349602788686752

